## Using neural networks to predict on Kryptonite-9 dataset

In [2]:
import numpy as np
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [3]:
print(torch.__version__)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Device: {0}'.format(device))

2.5.1+cu118
Device: cuda


In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_layer_stack = nn.Sequential(
            nn.Linear(9, 5),
            nn.ReLU(),
            nn.Linear(5,5), 
            nn.ReLU(),
            nn.Linear(5,1),
        ) 

    def forward(self, x):
        return self.linear_layer_stack(x)

In [ ]:
n = 9
X = np.load('Datasets/kryptonite-%s-X.npy'%(n))
y = np.load('Datasets/kryptonite-%s-y.npy'%(n))

# Shuffle and split the data
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.6, random_state=42)  # 60% training
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)  # 20% validation, 20% test

train_size = len(X_train)

model = NeuralNetwork()
model = model.to(device)
params = model.parameters()

optimiser = torch.optim.SGD(params, lr = 0.001)

criterion = torch.nn.BCELoss()

num_iter = 10

for epoch in range(num_iter):
    model.train()

    for point in range(train_size):
        datapoint = X_train[point]
        
        label = y_train[point]
        logit = model(torch.from_numpy(datapoint))
        loss = criterion(logit, label)
        print(f"Loss: {loss}")
        optimiser.zero_grad()
        loss.backward()
        optimiser.step()






RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument mat1 in method wrapper_CUDA_addmm)